<a href="https://colab.research.google.com/github/rammeshulam/appeal_consultant_demo/blob/main/appeal_consultant_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Israel Supreme Court - Appeals exploration
## Method:

1. Explore & prepare data

2. Define desired schema & Prompt to process appeal

3. Apply prompt and Schema on each appeal, store in BigQuery

4. Create an interactive report on top of BQ table

Bonus: Use aggregated insights to consult on future appeals


In [ ]:
import pandas as pd
import re
import pprint
import json
import pandas_gbq as gbq
import json

from google.colab import userdata
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
DRIVE_DIR = userdata.get('DRIVE_DIR') #caching large output files in google drive
projectid = 'ram-playground-420909'

project_dir = './'

# read main crime categories from file and create an enum accordingly:
crimes_categories_file = project_dir + 'crime_categories.txt'
with open(crimes_categories_file) as f:
  crime_categories = [line.rstrip('\n') for line in f]


prompt_path = project_dir + 'prompt.txt'
with open(prompt_path) as f:
  prompt = f.read()

#Focus on appeals of criminal cases (פלילי)
meta_inyan_nm = 'ע"פ'

#Clean dataset for data processing
verdicts_crime_clean_file_name = 'verdicts_crime_clean.csv'
verdicts_crime_clean_path = project_dir + verdicts_crime_clean_file_name
verdicts_crime_clean_cache_path = DRIVE_DIR + verdicts_crime_clean_file_name

#from google.colab import auth
#auth.authenticate_user()

# 1. Explore & prepare data



## 1.1 Read data

In [ ]:
df = pd.read_parquet("hf://datasets/LevMuchnik/SupremeCourtOfIsrael/cases_all.parquet")
df

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


,case_id,download_time,number_of_case_documents,file_name,Id,CaseId,VerdictDt,CreatedDate,CaseNum,CaseDesc,...,meta_ProgId,meta_is_technical,meta_judge_nm_last,meta_case_nbr,meta_verdict_ty,meta_lawyer_nm,meta_judge_nm_first,meta_verdict_pages,meta_inyan_nm,meta_court_nm
0,1,2022-09-17 14:58:23,8,HebrewVerdicts\03\560\058\07V\03058560_07V.txt,1,1,1920-11-23 20:00:00,2008-02-28 00:50:03.000,5856,"בג""ץ 5856/03",...,None,false,"[דורנר, נאור, חיות]","בג""ץ 5856/03",פסק-דין,"[ניצן שי, פרקליטות המדינה]","[דליה, מרים, אסתר]",2.0,"בג""ץ",בית המשפט העליון
1,1,2022-09-17 14:58:23,8,HebrewVerdicts\03\560\058\V03\03058560_V03.txt,159588,1,2003-07-08 19:00:00,2008-02-27 02:31:14.000,5856,"בג""ץ 5856/03",...,None,true,[חיות],"בג""ץ 5856/03",החלטה,"[ניצן שי, פרקליטות המדינה]",[אסתר],1.0,"בג""ץ",בית המשפט העליון
2,1,2022-09-17 14:58:23,8,HebrewVerdicts\03\560\058\V08\03058560_V08.txt,160618,1,2003-11-30 20:00:00,2008-02-27 02:31:14.000,5856,"בג""ץ 5856/03",...,None,true,"[דורנר, נאור, חיות]","בג""ץ 5856/03",החלטה,"[ניצן שי, פרקליטות המדינה]","[דליה, מרים, אסתר]",1.0,"בג""ץ",בית המשפט העליון
3,1,2022-09-17 14:58:23,8,HebrewVerdicts\03\560\058\V02\03058560_V02.txt,168038,1,2003-07-02 19:00:00,2008-02-27 02:31:13.000,5856,"בג""ץ 5856/03",...,None,true,[חיות],"בג""ץ 5856/03",החלטה,"[ניצן שי, פרקליטות המדינה]",[אסתר],1.0,"בג""ץ",בית המשפט העליון
4,1,2022-09-17 14:58:23,8,HebrewVerdicts\03\560\058\K05\03058560_K05.txt,168411,1,2003-10-15 20:00:00,2008-02-27 02:31:12.000,5856,"בג""ץ 5856/03",...,None,true,[שחם],"בג""ץ 5856/03",החלטה,"[ניצן שי, פרקליטות המדינה]",[עודד],1.0,"בג""ץ",בית המשפט העליון
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
751189,252705,2022-09-22 12:41:40,1,HebrewVerdicts\22\360\063\o03\22063360.O03,743252,252705,2022-09-20 21:00:00,2022-09-21 17:00:53.790,6336,"בש""פ 6336/22",...,Word.Document,None,None,None,None,None,None,NaN,None,None
751190,252707,2022-09-22 12:41:40,1,HebrewVerdicts\22\380\063\o01\22063380.O01,743254,252707,2022-09-20 21:00:00,2022-09-21 17:16:40.780,6338,"בש""פ 6338/22",...,Word.Document,None,None,None,None,None,None,NaN,None,None
751191,252721,2022-09-22 12:41:41,1,HebrewVerdicts\22\520\063\o01\22063520.O01,743242,252721,2022-09-20 21:00:00,2022-09-21 16:32:24.490,6352,"בש""פ 6352/22",...,Word.Document,None,None,None,None,None,None,NaN,None,None
751192,252724,2022-09-22 12:41:42,1,HebrewVerdicts\22\550\063\o01\22063550.O01,743258,252724,2022-09-20 21:00:00,2022-09-22 06:15:50.740,6355,"בש""פ 6355/22",...,Word.Document,None,None,None,None,None,None,NaN,None,None


started off with 751,194 rows

## 1.2 keep only verdics of criminal cases (exlucde decisions and others)
### Remove non-verdict rows

In [ ]:
df = df[df.TypeCode==2].copy()
df.shape

(84339, 44)

~11% (84,339 rows) remained

## 1.3 Remove null or short verdicts

In [ ]:
# clean extra spaces and "\n":
import re
df.text = df.text.str.replace('[\s|\\n]{2,}', '\n', regex=True)

#remove null or short verdicts:
df = df[df.text.str.len()>100]
df.shape

(83755, 44)

Out of all types of [cases](https://www.gov.il/BlobFolder/generalpage/net_law_info2/he/net_filelist.pdf) how many are

ערעור פלילי (עפ)


In [ ]:
df.meta_inyan_nm.value_counts()

,count
meta_inyan_nm,
"בג""ץ",11126
"ע""א",6071
"ע""פ",5158
"רע""א",1275
"עע""ם",575
"רע""פ",455
"על""ע",230
"בש""א",192
"רע""ב",164


we will focus on ערעור פלילי

In [ ]:
df = df[df.meta_inyan_nm==meta_inyan_nm].copy()
df.shape

(5158, 44)

## 1.4 Handling multi-row cases
### How many cases have more than one 'Psak Din' row?

In [ ]:
#How many cases have more than one 'Psak Din' row?
df.value_counts('CaseId').value_counts()

,count
count,
1,5068
2,42
3,2


Most cases have a single entry. For simplicity, let's drop multi-line cases altogether

In [ ]:
#Drop duplicates
df.drop_duplicates(subset='CaseId', keep=False, inplace=True)
df.shape

(5068, 44)

## 1.5 Removing non-hebrew cases and corrupted texts

In [ ]:
!pip install langdetect
from langdetect import detect
def detect_lang(text):
  try:
    return detect(text)
  except:
    return None
df['meta_lang'] = df['text'].apply(lambda x: detect_lang(x))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993221 sha256=8d3da5ff3de34818273d40eeead4080196d22b329f5606d781fa9aab6c579370
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [ ]:
df['meta_lang'].value_counts()

,count
meta_lang,
he,5068


In [ ]:
#df.to_csv(verdicts_crime_clean_path)

#df = pd.read_csv(verdicts_crime_clean_path)
df = pd.read_csv(verdicts_crime_clean_cache_path)
df.shape

(5068, 46)

In [ ]:
#df.columns
#df.dtypes
df_tbl = df[['CaseId','text','VerdictsDt']].copy()
df_tbl['summary'] = 'n/a'

In [ ]:
gbq.to_gbq(df_tbl, 'appeal_consultant_demo.verdicts', projectid, if_exists='replace') #'fail' 'replace'

100%|██████████| 1/1 [00:00<00:00, 6978.88it/s]


In [ ]:
#%%bigquery --project ram-playground-420909
#select * from appeal_consultant_demo.verdicts limit 1

Seems all remaining rows are hebrew...

# 2. Define desired schema & Prompt to process appeal

## Create model and schema for response

In [ ]:
from google.ai.generativelanguage_v1beta.types import content

crime_categories_schema = content.Schema(type = content.Type.STRING,
                                         format = 'enum',
                                         enum = crime_categories)

In [ ]:
"""
Install the Google AI Python SDK

$ pip install google-generativeai
$ pip install google.ai.generativelanguage
"""

import os
import google.generativeai as genai
from google.ai.generativelanguage_v1beta.types import content
from google.generativeai.types import HarmCategory, HarmBlockThreshold

genai.configure(api_key=GEMINI_API_KEY)

# Create the model
generation_config = {
  "temperature": 0,
  "max_output_tokens": 8192,
  "response_schema": content.Schema(
    type = content.Type.OBJECT,
    properties = {
      "response": content.Schema(
        type = content.Type.OBJECT,
        properties = {
          "summary": content.Schema(
            type = content.Type.STRING,
          ),
          "crime_categories": content.Schema(
              type = content.Type.ARRAY,
              items = crime_categories_schema,
              description = "סיווג ההאשמות בתיק לקטגוריה (בין 1-3 קטגוריות, מסודרות לפי חשיבות)"
          ),
          "reason_for_appeal": content.Schema(
            type = content.Type.STRING,
            description = "Why is the case being raised? for appeal"
          ),
          "is_punishment_does_not_match_crime": content.Schema(
            type = content.Type.BOOLEAN,
            description = "האם הערעור טוען שהעונש אינו תואם לחומרת העבירה?"
          ),
          "is_extenuating_circumstances": content.Schema(
            type = content.Type.BOOLEAN,
            description = "האם הערעור מבקש להתחשב בנסיבות מיוחדות בהן בוצע הפשע?"
          ),
          "is_personal_circumstances": content.Schema(
            type = content.Type.BOOLEAN,
            description = "האם הערעור מבקש להתחשב בנסיבות המיוחדות של הנאשם?"
          ),
          "is_new_evidence": content.Schema(
            type = content.Type.BOOLEAN,
            description = "האם הערעור נובע מראיות או עדויות חדשות שעלו מאז גזר הדין?"
          ),
          "sections_of_Penal_Code" : content.Schema(
            type = content.Type.STRING,
            description = "סעיפים מחוק העונשין המופיעים בערעור"
          ),
          "punishment": content.Schema(
            type = content.Type.STRING,
            description = "גזר הדין בערעור"
          ),
        },
      ),
    },
  ),
  "response_mime_type": "application/json",
}

def get_model():
  model = genai.GenerativeModel(
    #model_name="gemini-1.5-flash",
    model_name="gemini-1.5-pro-latest",
    #model_name="gemini-1.5-pro-exp-0801",
    #model_name="gemini-1.5-pro",
    generation_config=generation_config,
    # See https://ai.google.dev/gemini-api/docs/safety-settings
    safety_settings={
          HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
          HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
          HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
          HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE
          }
    )
  return model

model = get_model()

In [ ]:
def show_response(response_str, CaseId):
  response_json = json.loads(response_str)
  response_json['CaseId'] = CaseId
  print(json.dumps(response_json, indent=2, ensure_ascii=False).encode('utf8').decode())

#json_object = json.loads(response.text)
#json_formatted_str = json.dumps(json_object, indent=2, ensure_ascii=False)
#print(json_formatted_str)

def summarize_case(row, model, prompt):
  chat_session = model.start_chat(
    history=[
    ]
  )
  response = chat_session.send_message(prompt + "\n\n" + row.text)
  return response

In [ ]:
from google.cloud import bigquery

client = bigquery.Client(project = projectid)

def update_bq_with_suammry(client, CaseId, summary):
  query = (
      """UPDATE appeal_consultant_demo.verdicts
      SET summary = @summary
      WHERE CaseId=@CaseId""")
  job_config = bigquery.QueryJobConfig(
      query_parameters=[
          bigquery.ScalarQueryParameter("summary", "STRING", summary),
          bigquery.ScalarQueryParameter("CaseId", "INT64", CaseId),
      ]
  )
  query_job = client.query(query, job_config=job_config)  # API request
  query_job.result()
  if(query_job.state=='DONE'):   # Waits for statement to finish
    return True
  return False

def fetch_ids_to_process(client):
  return client.query("select CaseId from appeal_consultant_demo.verdicts where summary='n/a'").to_dataframe().CaseId.tolist()
#update_bq_with_suammry(client, 59205, "n/a")
#fetch_ids_to_process(client)

True

## Load prompt

In [ ]:
pprint.pp(prompt)

('להלן ערעור/בקשת רשות לערעור על פסק דין. ברצוני לקבל:\n'
 ' * סיכום קצר של התיק.\n'
 ' \n'
 ' * סיב(ו)ת הערעור.\n'
 ' * האם הערעור התקבל\n'
 ' * סעיפים לחוק העונשין המוזכרים בתיק\n'
 ' * גזר הדין שהתקבל - חודשי מאסר, מאסר על תנאי, קנס כספי, עבודות שירות וכו׳\n'
 ' \n'
 ' * התייחסות ספציפית לסיבות הבאות (התשובה היא כן/לא עבור כל סעיף):\n'
 ' - האם הערעור מתייחס לאי התאמה לחומרת העבירה, \n'
 ' - האם הערעור מתייחס לחוסר התחשבות בנסיבות ביצוע העבירה\n'
 ' - בקשה להתחשבות בנסיבותיו האישיות של הנאשם\n'
 ' - האם הערעור הוא לאור ראיות/עדויות חדשות\n'
 ' \n'
 ' * סיווג התיק לאחת או יותר מהקטגוריות הבאות:\n'
 ' עבירות נגד האדם:\n'
 ' עבירות אלימות: כוללות רצח, הריגה, חבלה בכוונה מחמירה, תקיפה הגורמת חבלה '
 'של \n'
 ' ממש, איומים.\n'
 ' עבירות מין: כוללות אינוס, מעשה מגונה, הטרדה מינית, בעילה אסורה בהסכמה, '
 'גילוי \n'
 ' עריות.\n'
 ' עבירות נגד החירות: כוללות חטיפה, כליאת שווא, סחר בבני אדם.\n'
 ' עבירות נגד הרכוש:\n'
 ' עבירות גניבה: כוללות גניבה, גניבה בידי מורשה, מעילה, שוד, התפרצות.\n'
 '

## Test on a sample case

In [ ]:
sample_CaseId = 109031
sample_case = df[df.CaseId==sample_CaseId].text.iloc[0]
pprint.pp(sample_case)

('\n'
 'בבית המשפט\n'
 'העליון\n'
 'בשבתו כבית משפט לערעורים פליליים\n'
 'ע"פ\n'
 '407/07\n'
 'בפני:\n'
 "כבוד השופט א' גרוניס\n"
 "כבוד השופטת א' חיות\n"
 "כבוד השופט ע' פוגלמן\n"
 'המערער:\n'
 'שאדי פארס\n'
 'נ\n'
 'ג\n'
 'ד\n'
 'המשיבה:\n'
 'מדינת ישראל\n'
 'ערעור על הכרעת הדין וגזר הדין\n'
 'שניתנו בבית המשפט המחוזי תל אביב מיום 20.12.06 בתיק פ 40088/06, על ידי כבוד\n'
 "השופט צ' גורפינקל\n"
 'תאריך הישיבה:\n'
 'כ"ז בשבט התשס"ח\n'
 '(03.02.08)\n'
 'בשם המערער:\n'
 'עו"ד יונס אחמד חמזה\n'
 'בשם המשיבה:\n'
 'עו"ד דפנה פינקלשטיין\n'
 'פסק-דין\n'
 "השופט א' גרוניס:\n"
 '1.\n'
 'המערער הורשע בעקבות שמיעת ראיות בעבירות\n'
 'של שוד בנסיבות מחמירות, היינו בחבורה, וכן בהסעה שלא כדין של שוהים בלתי '
 'חוקיים.\n'
 "בית המשפט המחוזי בתל אביב (כבוד השופט צ' גורפינקל) גזר למערער 30 חודשי מאסר\n"
 'בגין עבירת השוד ו-4 חודשי מאסר בגין העבירה האחרת. העונש האחרון הוטל בחופף\n'
 'לראשון. כמו כן הופעל במצטבר מאסר מותנה של 18 חודשים. בנוסף לכך נגזרו שני '
 'מאסרים\n'
 'מותנים. התוצאה היא שעל המערער לרצ

In [ ]:
chat_session = model.start_chat(
  history=[
  ]
)

response = chat_session.send_message(prompt + "\n\n" + sample_case)
show_response(response.text, sample_CaseId)

{
  "response": {
    "crime_categories": [
      "גניבה/שוד",
      "אלימות/איומים"
    ],
    "is_extenuating_circumstances": true,
    "is_new_evidence": false,
    "is_personal_circumstances": true,
    "is_punishment_does_not_match_crime": true,
    "punishment": "בית המשפט קיבל את הערעור באופן חלקי וקיצר את עונש המאסר של המערער מ-4 שנים ל-39 חודשים.",
    "reason_for_appeal": "הערעור הוגש על עצם הפעלתו של מאסר על תנאי של 18 חודשים, ולחלופין, על משך הפעלתו. בנוסף, נטען כי העונש שהוטל על המערער חמור יתר על המידה.",
    "sections_of_Penal_Code": "382(א)",
    "summary": "המערער הורשע בשוד בנסיבות מחמירות ובהסעה שלא כדין של שוהים בלתי חוקיים. בית המשפט המחוזי גזר עליו 30 חודשי מאסר בגין השוד ו-4 חודשי מאסר בגין העבירה האחרת, וכן הפעיל מאסר מותנה של 18 חודשים במצטבר. בערעורו טען המערער כי לא היה מקום להפעיל את המאסר המותנה, ולחלופין, כי היה עליו להיות מופעל בחופף. כמו כן, טען כי העונש שהוטל עליו חמור יתר על המידה. בית המשפט העליון קיבל את הערעור באופן חלקי וקיצר את עונש המאסר ל-39 חוד

## Now a test on a sample batch

In [ ]:
for index, row in df.head(2).iterrows():
  print(row.CaseId)
  response = summarize_case(row, model, prompt)
  show_response(response.text, row.CaseId)

19
{
  "response": {
    "crime_categories": [
      "אלימות/חבלה בכוונה מחמירה",
      "מין/אינוס"
    ],
    "is_extenuating_circumstances": false,
    "is_new_evidence": false,
    "is_personal_circumstances": false,
    "is_punishment_does_not_match_crime": true,
    "punishment": "22 שנות מאסר",
    "reason_for_appeal": "העונש שהוטל על המערער בגין עבירות אלימות קשות ואינוס כלפי אשתו הוא מחמיר יתר על המידה.",
    "sections_of_Penal_Code": "329(1), 345(ב)(3), 345(ב)(4), 192",
    "summary": "ערעור על חומרת העונש שהוטל על המערער, עופר לוי, לאחר שהורשע בבית המשפט המחוזי בתל אביב בעבירות של חבלה בכוונה מחמירה, אינוס תוך התעללות ואיומים כלפי אשתו. בית המשפט המחוזי גזר על המערער 22 שנות מאסר. המערער טוען כי העונש מחמיר יתר על המידה."
  },
  "CaseId": 19
}
20
{
  "response": {
    "crime_categories": [
      "אלימות/רצח",
      "אלימות/הריגה"
    ],
    "is_extenuating_circumstances": false,
    "is_new_evidence": false,
    "is_personal_circumstances": true,
    "is_punishment_does_not_m